In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from pathlib import Path

def download_one_episode(title,no,url):
    
    req_header = {'referer': url}
    
    res = requests.get(url)
    print(res.ok)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
               
        img_tags = soup.select("img[src*='IMAG01']")
        imgurl_list = [img['src'] for img in img_tags]
        print(len(imgurl_list))
        
        save_dir = Path('img') / title / str(no)
        save_dir.mkdir(parents=True, exist_ok=True)

        for idx,img_url in enumerate(imgurl_list,1):
            res = requests.get(img_url,headers=req_header)
            res.raise_for_status()

            file_name = Path(img_url).name
            save_path = save_dir / file_name
            save_path.write_bytes(res.content)
            print(f'다운로드 완료: {save_path} ({save_path.stat().st_size:,} bytes)')


[]


In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from urllib.parse import urlparse, parse_qs

def download_all_episode(title,episode_url):
    #url 을 파싱해서 titleId를 알아내기
    parsed_url = urlparse(episode_url)
    query_params = parse_qs(parsed_url.query) #딕셔너리 값은 리스트로 ex) {'query': ['python'], 'lang': ['ko']}
    title_id = query_params.get('titleId', [''])[0] #'titleId'가 존재하지 않을 경우 기본값으로 [''](빈 리스트)를 반환하도록 설정

    api_url = f'https://comic.naver.com/api/article/list?titleId={title_id}&page=1'
               #https://comic.naver.com/webtoon/detail?titleId=826419&no=46
    res = requests.get(api_url)
    print(res.status_code)    
    if res.ok:
        #pprint(res.json()['articleList'])
        for article in res.json()['articleList']:
            no = article['no']
            detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
            print(detail_url)
        

if __name__ == '__main__': 
    download_all_episode('롤플레잉','https://comic.naver.com/webtoon/list?titleId=826419')

200
https://comic.naver.com/webtoon/detail?titleId=826419&no=46
https://comic.naver.com/webtoon/detail?titleId=826419&no=45
https://comic.naver.com/webtoon/detail?titleId=826419&no=44
https://comic.naver.com/webtoon/detail?titleId=826419&no=43
https://comic.naver.com/webtoon/detail?titleId=826419&no=42
https://comic.naver.com/webtoon/detail?titleId=826419&no=41
https://comic.naver.com/webtoon/detail?titleId=826419&no=40
https://comic.naver.com/webtoon/detail?titleId=826419&no=39
https://comic.naver.com/webtoon/detail?titleId=826419&no=38
https://comic.naver.com/webtoon/detail?titleId=826419&no=37
https://comic.naver.com/webtoon/detail?titleId=826419&no=36
https://comic.naver.com/webtoon/detail?titleId=826419&no=35
https://comic.naver.com/webtoon/detail?titleId=826419&no=34
https://comic.naver.com/webtoon/detail?titleId=826419&no=33
https://comic.naver.com/webtoon/detail?titleId=826419&no=32
https://comic.naver.com/webtoon/detail?titleId=826419&no=31
https://comic.naver.com/webtoon/deta

In [ ]:
from urllib.parse import urlparse, parse_qs

"""url에 titleId를 반환하는 함수"""
def get_title_id(url):
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    title_id = query_params.get('titleId', [''])[0]
    return title_id

#테스트 하기
url = 'https://comic.naver.com/webtoon/list?titleId=826419'
print(get_title_id(url))  # 출력: 826419

In [2]:
def calculate_pages(total_items, items_per_page=20):
    """총 페이지 수 계산 함수"""
    return (total_items + items_per_page - 1) // items_per_page

# 예제 사용
total_items = 49
items_per_page = 20

total_pages = calculate_pages(total_items, items_per_page)
print(f"총 {total_items}개의 항목을 {items_per_page}개씩 출력할 때 필요한 페이지 수: {total_pages}")
# 출력: 총 49개의 항목을 20개씩 출력할 때 필요한 페이지 수: 3

총 49개의 항목을 20개씩 출력할 때 필요한 페이지 수: 3


In [ ]:
import requests

def download_all_episode(title,episode_url):
    title_id = get_title_id(episode_url)

    ajax_url = f'https://comic.naver.com/api/article/list?titleId={title_id}'               
    res = requests.get(ajax_url)

    if res.ok:
        total_count = res.json()['totalCount']
        for count in range(calculate_pages(total_count)):
            #count 변수 0,1,2 // page 번호는 1,2,3
            page = count + 1
            req_param = { "page": page}
            print(req_param)
            res = requests.get(ajax_url, params=req_param)
            for article in res.json()['articleList']:
                no = article['no']
                detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
                print(detail_url)
        

if __name__ == '__main__': 
    download_all_episode('롤플레잉','https://comic.naver.com/webtoon/list?titleId=826419')

In [ ]:
import requests
from time import sleep

def download_all_episode(title,episode_url):
    title_id = get_title_id(episode_url)

    ajax_url = f'https://comic.naver.com/api/article/list?titleId={title_id}'               
    res = requests.get(ajax_url)

    if res.ok:
        total_count = res.json()['totalCount']
        for count in range(calculate_pages(total_count)):
            page = count + 1
            req_param = { "page": page}
            print(req_param)
            res = requests.get(ajax_url, params=req_param)
            for article in res.json()['articleList']:
                no = article['no']
                detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
                print(detail_url)
                download_one_episode(title,no,detail_url)
                #0.5초간 프로세스를 중지함, 기계가 아니라 사람처럼 보이게 하려고
                sleep(0.5)

if __name__ == '__main__': 
    #download_all_episode('롤플레잉','https://comic.naver.com/webtoon/list?titleId=826419')
    download_all_episode('냉동무사','https://comic.naver.com/webtoon/list?titleId=836370')